In [2]:
# Cell 1: importy i konfiguracja ścieżek
from pathlib import Path

# jeśli używasz struktury "src", jak w obecnym notebooku:
import sys

src_path = Path("src").resolve()
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from aml_data_generator import (
    DatasetSpec,
    generate_default_datasets,
    generate_transactions_csv,
    write_demo_transactions_csv,
)

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

print(f"📂 Katalog danych: {data_dir.resolve()}")


📂 Katalog danych: /home/jupyter/olga_zydziak/version_beta/Folder/casual_model/Axiomatic-learning/data


In [3]:
# Cell: Generator syntetycznych danych AML z kontrolą liczby rekordów i udziałów scenariuszy

from pathlib import Path
from aml_data_generator import AMLDataGenerator, AMLScenarioWeights

# Katalog na dane – używamy istniejącego data_dir, a jeśli go nie ma, tworzymy
if "data_dir" not in globals():
    data_dir = Path("data")
    data_dir.mkdir(exist_ok=True)
else:
    data_dir.mkdir(exist_ok=True)

# === PARAMETRY DO EDYCJI ===

# Ile rekordów chcesz wygenerować
N_RECORDS = 2000  # <--- zmieniasz tutaj

# Jakie udziały scenariuszy (dowolne liczby dodatnie, generator sam je znormalizuje)
scenario_weights = AMLScenarioWeights(
    clean=0.55,          # „zwykłe” transakcje
    high_amount=0.15,    # wysokie kwoty
    velocity=0.15,       # duża liczba transakcji
    pep_high_risk=0.10,  # PEP-y / wyższe ryzyko
    structuring=0.03,    # „structuring” – wiele małych, blisko progu
    noise=0.02,          # szum / dziwne mixy
)

# === GENEROWANIE DANYCH ===

generator = AMLDataGenerator(
    seed=10,  # zmień, jeśli chcesz inne losowanie przy tych samych wagach
    scenario_weights=scenario_weights,
)

output_path = data_dir / "transactions_aml_compare_dataset.csv"
output_path = generator.generate_csv(path=output_path, n_records=N_RECORDS)

# Obliczamy znormalizowane udziały scenariuszy (tak, jak widzi je generator)
total_weight = (
    scenario_weights.clean
    + scenario_weights.high_amount
    + scenario_weights.velocity
    + scenario_weights.pep_high_risk
    + scenario_weights.structuring
    + scenario_weights.noise
)

print("📁 Wygenerowano syntetyczny zbiór AML:")
print(f"- Plik:            {output_path}")
print(f"- Liczba rekordów: {N_RECORDS}\n")

print("🎛 Zadane udziały scenariuszy (po normalizacji):")
for name in (
    "clean",
    "high_amount",
    "velocity",
    "pep_high_risk",
    "structuring",
    "noise",
):
    raw = getattr(scenario_weights, name)
    share = (raw / total_weight) if total_weight else 0.0
    print(f"  - {name:14s}: {raw:.3f} → ok. {share:.1%} rekordów")

print(
    "\nℹ️ Uwaga: generator losuje scenariusze według tych udziałów, "
    "więc rzeczywiste procenty w pliku mogą się minimalnie różnić."
)


📁 Wygenerowano syntetyczny zbiór AML:
- Plik:            data/transactions_aml_compare_dataset.csv
- Liczba rekordów: 2000

🎛 Zadane udziały scenariuszy (po normalizacji):
  - clean         : 0.550 → ok. 55.0% rekordów
  - high_amount   : 0.150 → ok. 15.0% rekordów
  - velocity      : 0.150 → ok. 15.0% rekordów
  - pep_high_risk : 0.100 → ok. 10.0% rekordów
  - structuring   : 0.030 → ok. 3.0% rekordów
  - noise         : 0.020 → ok. 2.0% rekordów

ℹ️ Uwaga: generator losuje scenariusze według tych udziałów, więc rzeczywiste procenty w pliku mogą się minimalnie różnić.
